# ASTRA DB VECTOR STORE

**Astra DB** is a **cloud-native database service** from **DataStax** that’s designed to make working with **Apache Cassandra** and **vector search** super easy — without you having to manage servers or clusters yourself.

It’s often used in **AI, RAG (Retrieval-Augmented Generation)**, and **search projects** because it can store both:

- **Structured data** (like a normal NoSQL database)

- **Unstructured data embeddings** (for semantic search)

**Key Features**  
**1. Fully managed Cassandra**  
You don’t install or manage nodes — Astra handles scaling, backups, and updates.

**2. Vector Search built-in**  
- You can store vector embeddings (from OpenAI, AWS Bedrock, NVIDIA, etc.) directly in the database.

- Supports semantic search (find similar meaning, not just keyword matches).

**3. Multi-provider embedding support**  
- Can generate embeddings server-side via:
 - OpenAI
 - AWS Bedrock (Amazon Titan, etc.)
 - NVIDIA (NV-Embed)
 - Cohere
 
**4. Flexible API access**  
- REST API
- gRPC
- GraphQL
- Client libraries (like langchain_astradb)

**5. Serverless**  
- Pay only for what you use — no pre-allocated hardware.

**6. Multi-region**  
- Deploy closer to your users for lower latency.

**Why AI people use Astra DB**  
- You can combine raw text + metadata + vector embeddings in one place.
- Perfect for RAG pipelines:
 - Take your documents.
 - Embed them.
 - Store in Astra DB.
 - Search by similarity when a user asks a question.

**Analogy**   
Think of Astra DB like Google Drive for AI data:
 - Stores the text content (like storing PDFs, articles).
 - Also stores a semantic fingerprint of each piece of text (vector embedding) so you can find by meaning, not just filename.

In [1]:
import os
from dotenv import load_dotenv

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [3]:
ASTRA_DB_API_ENDPOINT = os.getenv("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_KEYSPACE = os.getenv("ASTRA_DB_KEYSPACE")  # optional

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_astradb import AstraDBVectorStore

# --- Step 1: Load PDF ---
pdf_path = "data\\attention-is-all-you-need-Paper.pdf"  # Your PDF file path
loader = PyPDFLoader(pdf_path)
docs = loader.load()

# --- Step 2: Split text into chunks ---
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_documents(docs)

print(f"Loaded {len(chunks)} chunks from PDF.")



Loaded 51 chunks from PDF.


## Method 1 – Explicit Embeddings

We embed the text locally using OpenAI and store it.

In [5]:
vector_store_explicit = AstraDBVectorStore(
    collection_name="pdf_explicit_embeddings",
    embedding=embeddings,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE
)

# Add PDF chunks to AstraDB
vector_store_explicit.add_documents(chunks)
print("Method 1: Stored using explicit embeddings.")


Method 1: Stored using explicit embeddings.


## Method 2 – Server-side Vectorization (vectorize)

Astra DB does the embeddings for you (no local OpenAI calls).
This requires that Astra DB is set up with a Vector Service (e.g., NVIDIA, OpenAI, Cohere).

In [ ]:
from astrapy.info import VectorServiceOptions
from langchain_astradb import AstraDBVectorStore

vs_options = VectorServiceOptions(
    provider="bedrock",                      # AWS Bedrock provider
    model_name="amazon.titan-embed-text-v1"   # Change to the exact model you enabled in Astra
)

vector_store_vectorize = AstraDBVectorStore(
    collection_name="pdf_server_vectorize_aws",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE,
    collection_vector_service_options=vs_options
)

# Add PDF chunks (server-side embedding via AWS Bedrock in Astra DB)
vector_store_vectorize.add_documents(chunks)
print("Stored using AWS Bedrock vector service in Astra DB.")


## Method 3 – Auto-detect from Existing Collection


Method 3 is the auto-detect from existing collection method for Astra DB.
This assumes:

You already have a collection in Astra DB (created earlier through Method 1, Method 2, or Astra’s UI/API).

You just want to connect to it and start adding/querying documents without redefining vector settings.

In [6]:
vector_store_autodetect = AstraDBVectorStore(
    collection_name="pdf_explicit_embeddings",  # Must match existing collection name in Astra
    embedding=embeddings,                       # Optional if vectors already exist
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE,
    autodetect_collection=True
)

c:\Users\viswa\Documents\GENAIandAGENTICAI\LangChain_Vector_Stores\vectorstore\Lib\site-packages\langchain_astradb\vectorstores.py:957: UserWarning: Collection supports lexical; however, autodetect encountered documents without a $lexical. The $lexical field will be set for new documents, but the pre-existing documents might not be reached by lexical search when running hybrid search mode.
  self.document_codec = _detect_document_codec(


In [7]:
# ------------------------
# Step 3: Test a search
# ------------------------
results = vector_store_autodetect.similarity_search("What is the paper about?", k=3)
for i, res in enumerate(results, start=1):
    print(f"{i}. {res.page_content[:200]}...\n")

c:\Users\viswa\Documents\GENAIandAGENTICAI\LangChain_Vector_Stores\vectorstore\Lib\site-packages\langchain_astradb\vectorstores.py:2677: BetaFeatureWarning: Method 'Collection.find_and_rerank' is in beta and might undergo signature or behaviour changes in the future.
  hybrid_reranked_results = self.astra_env.collection.find_and_rerank(


1. Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz...

2. [32] Jie Zhou, Ying Cao, Xuguang Wang, Peng Li, and Wei Xu. Deep recurrent models with
fast-forward connections for neural machine translation. CoRR, abs/1606.04199, 2016.
11...

3. mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine transla...

